In [1]:
import pandas as pd
import chromadb
import random
from langchain.text_splitter import RecursiveCharacterTextSplitter

Saving JDs

In [17]:
file_path = '../talenttrove/data/jobs/mycareersfuture_with_descriptions_cleaned.json'
with open(file_path, 'r') as file:
    df = pd.read_json(file, orient='records')

df.head()

,company_name,company_url,company_logo,uen,employeeCount,positionLevels,skills,employmentTypes,status,updatedAt,jobPostId,title,salary_min,salary_max,salary_type,description
0,MINDFLEX EDUCATION PTE. LTD.,https://singaporetuitionteachers.com,https://static.mycareersfuture.gov.sg/images/c...,202206896K,NaN,[Fresh/entry level],"[Microsoft PowerPoint, Microsoft Office, Micro...","[Contract, Flexi-work, Freelance, Full Time, I...",Open,2023-10-26T10:14:11.000Z,MCF-2023-0817909,Tuition Coordinator - Admin Support & Sales,3700,2500,Monthly,Find Out Even More At: https://tinyurl.com/min...
1,SATS FOOD SERVICES PTE. LTD.,None,None,197300678G,NaN,[Fresh/entry level],"[Tape, Television, Microsoft Excel, Advertisin...","[Permanent, Full Time]",Re-open,2023-10-16T12:53:17.000Z,MCF-2023-0693756,Auction Assistant,2263,1518,Monthly,Job Description:\n\nYour scope requires you to...
2,EQUIP GLOBAL PTE. LTD.,None,https://static.mycareersfuture.gov.sg/images/c...,201306854R,NaN,[Fresh/entry level],"[Market Research, Ability to Multitask, Manage...","[Permanent, Full Time]",Open,2023-10-11T14:26:08.000Z,MCF-2023-0761746,Event Planner,3200,3100,Monthly,The Event Planner (Conference Producer) is cri...
3,INITIA MANAGEMENT PTE. LTD.,None,None,202102387G,NaN,[Fresh/entry level],"[Recipes, Lifestyle, Ingredients, Food Safety,...","[Permanent, Full Time]",Re-open,2023-10-17T10:09:50.000Z,MCF-2023-0653055,Kitchen Assistant,2700,1800,Monthly,Responsibilities\n\nEnsure all food and other ...
4,RECRUIT EXPRESS PTE LTD,https://www.recruitexpress.com.sg/,https://static.mycareersfuture.gov.sg/images/c...,199601303W,NaN,[Fresh/entry level],"[Microsoft Office, Microsoft Excel, Handle cal...","[Contract, Part Time, Permanent, Full Time]",Open,2023-10-21T18:40:52.000Z,MCF-2023-0804957,Call Centre Support Officer / Customer Service...,2500,1500,Monthly,Job Description:\n\nTo receive and make inboun...


In [27]:
documents = []
metadatas = []
ids = []

for index, row in df.iterrows():
    company_name = row['company_name']
    uen = row['uen']
    job_post_id = row['jobPostId']

    documents.append(row['description'])
    metadatas.append({'company_name': company_name, 'uen': uen, 'job_post_id': job_post_id})
    ids.append(str(index))

Creating the a ChromaDB vector database, and a collection for the MyCareerFuture JDs


In [2]:
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.PersistentClient(path="jd_vectordb")

In [3]:
collection = chroma_client.get_or_create_collection(name="mycareersfuture_jd")

In [ ]:
batch_size = 500  # Adjust this value based on the maximum batch size allowed
for i in range(0, len(documents), batch_size):
    batch_documents = documents[i:i+batch_size]
    batch_metadatas = metadatas[i:i+batch_size]
    batch_ids = ids[i:i+batch_size]

    # Ensure that the order of documents, metadatas, and ids is maintained within each batch
    assert len(batch_documents) == len(batch_metadatas) == len(batch_ids)

    # Add the batch to the collection
    collection.add(
        documents=batch_documents,
        metadatas=batch_metadatas,
        ids=batch_ids
    )

In [4]:
import pickle
with open('generated_jd_test.pkl', 'rb') as file:
    sample_text = pickle.load(file)

sample_text

'We are seeking a results-oriented and experienced Real Estate Agent to join our team. As a Real Estate Agent, you will be responsible for residential and commercial property sales, exceeding sales targets, and providing exceptional customer service. You will utilize your expertise in market analysis, property marketing, and negotiation to guide clients through real estate transactions.\n\nJob Responsibilities:\n- Exceed annual sales targets by utilizing your market knowledge and expertise in property valuation and neighborhood trends.\n- Conduct detailed market analysis to determine optimal pricing strategies for listings, resulting in reduced time on market.\n- Lead marketing efforts by creating high-quality listing presentations, virtual tours, and social media campaigns.\n- Successfully negotiate contracts, resolve issues, and facilitate smooth transactions between buyers and sellers.\n- Build and maintain strong client relationships through buyer/seller consultations, needs analys

In [25]:
results = collection.query(
    query_texts=[sample_text],
    n_results=1,
    include=['documents', 'distances', 'metadatas']
)

results

{'ids': [['1820']],
 'distances': [[0.7106396555900574]],
 'metadatas': [[{'company_name': 'BIZSAVER PTE. LTD.',
    'job_post_id': 'MCF-2023-0813023',
    'uen': '201621245M'}]],
 'embeddings': None,
 'documents': [["As a Mortgage and Real Estate Specialist, you will be responsible for the growth of BizSaver's Mortgage and Real Estate business while providing financing solutions to meet the needs of our clients.\n\nYou will assigned leads and properties to market, negotiate and close. Training will be provided to improve your professional knowledge and efficiency.\n\nYou will identify and maximise sales opportunities, proactively build and manage long-term customer relationships with an emphasis on excellent, ethical, authentic and reliable service.\n\nYou will be provided opportunities to network for deeper engagement with our business partners, including Lawyers, Bankers and other complementing partners. \n\nApart from exceeding BizSaver and your personal targets, you will work clos

In [27]:
# Most similar result
most_similar_jd = results['documents'][0][0]
most_similar_jd

"As a Mortgage and Real Estate Specialist, you will be responsible for the growth of BizSaver's Mortgage and Real Estate business while providing financing solutions to meet the needs of our clients.\n\nYou will assigned leads and properties to market, negotiate and close. Training will be provided to improve your professional knowledge and efficiency.\n\nYou will identify and maximise sales opportunities, proactively build and manage long-term customer relationships with an emphasis on excellent, ethical, authentic and reliable service.\n\nYou will be provided opportunities to network for deeper engagement with our business partners, including Lawyers, Bankers and other complementing partners. \n\nApart from exceeding BizSaver and your personal targets, you will work closely as a reliable team player and take charge of your own time planning sales visits and presentations to potential customers.\n\nThis role encompasses engagement via cold calling and texting with potential business p

Creating collection for generated JDs, and the corresponding MyCareersFuture JD

In [29]:
collection2 = chroma_client.get_or_create_collection(name="generated_jd")

documents2 = []
metadatas2 = []
ids2 = []
idx = 0

documents2.append(sample_text)
metadatas2.append({'most_similar_jd': most_similar_jd})
ids2.append(str(idx))

collection2.add(
        documents=documents2,
        metadatas=metadatas2,
        ids=ids2
    )



In [30]:
collection2.get(idx)

{'ids': ['0'],
 'embeddings': None,
 'metadatas': [{'most_similar_jd': "As a Mortgage and Real Estate Specialist, you will be responsible for the growth of BizSaver's Mortgage and Real Estate business while providing financing solutions to meet the needs of our clients.\n\nYou will assigned leads and properties to market, negotiate and close. Training will be provided to improve your professional knowledge and efficiency.\n\nYou will identify and maximise sales opportunities, proactively build and manage long-term customer relationships with an emphasis on excellent, ethical, authentic and reliable service.\n\nYou will be provided opportunities to network for deeper engagement with our business partners, including Lawyers, Bankers and other complementing partners. \n\nApart from exceeding BizSaver and your personal targets, you will work closely as a reliable team player and take charge of your own time planning sales visits and presentations to potential customers.\n\nThis role encomp

In [34]:
# Test: Querying a collection of a database using an item from another collection of the database
results2 = collection.query(
    query_texts=collection2.get(idx)['documents'],
    n_results=1,
    include=['documents', 'distances', 'metadatas']
)

results2 # Results are the same as earlier, meaning the implementation of the generated JD collection is correct

{'ids': [['1820']],
 'distances': [[0.7106396555900574]],
 'metadatas': [[{'company_name': 'BIZSAVER PTE. LTD.',
    'job_post_id': 'MCF-2023-0813023',
    'uen': '201621245M'}]],
 'embeddings': None,
 'documents': [["As a Mortgage and Real Estate Specialist, you will be responsible for the growth of BizSaver's Mortgage and Real Estate business while providing financing solutions to meet the needs of our clients.\n\nYou will assigned leads and properties to market, negotiate and close. Training will be provided to improve your professional knowledge and efficiency.\n\nYou will identify and maximise sales opportunities, proactively build and manage long-term customer relationships with an emphasis on excellent, ethical, authentic and reliable service.\n\nYou will be provided opportunities to network for deeper engagement with our business partners, including Lawyers, Bankers and other complementing partners. \n\nApart from exceeding BizSaver and your personal targets, you will work clos